In [3]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import glob
import os
from ripple_heterogeneity.utils import functions

functions.set_plotting_defaults()
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
fig_save_path = r'Z:\home\ryanh\projects\ripple_heterogeneity\figures\panels'

In [10]:
files = glob.glob(r"Z:/home/Heath/Viral_tracing/CTB/**/*.csv",recursive=True)
print(files)
pd.read_csv(files[0])

['Z:/home/Heath/Viral_tracing/CTB\\CTBR1\\day3\\18\\Results.csv']


,,Area,Mean,Min,Max,X,Y,Ch,Slice
0,1,0.0,4013,4013,4013,591.482,162.845,1,3
1,2,0.0,2094,2094,2094,272.968,183.538,1,5
2,3,0.0,2127,2127,2127,300.420,168.564,1,5
3,4,0.0,2658,2658,2658,338.688,165.236,1,5
4,5,0.0,1617,1617,1617,501.740,161.077,1,6
5,6,0.0,3227,3227,3227,835.333,165.236,1,4
6,7,0.0,4281,4281,4281,883.583,169.396,1,4
7,8,0.0,5468,5468,5468,514.219,160.245,1,1
8,9,0.0,4743,4743,4743,535.848,155.254,1,1
9,10,0.0,3627,3627,3627,481.775,148.598,1,1
